In [2]:
import tensorflow as tf

2024-05-27 15:33:28.829771: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-27 15:33:28.929315: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-27 15:33:28.929368: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-27 15:33:28.945746: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-27 15:33:28.989021: I tensorflow/core/platform/cpu_feature_guar

In [3]:
class ConvBlock2D(tf.keras.layers.Layer):
    """
    Description:
    This class implemenents the basic convolution block of the architecture.
    It consists of an input 'Conv2D[1x1]-BatchNorm-ReLU' block followed by a number
    of hidden, 'Conv2D[nxn]-BatchNorm-ReLU' blocks. Residual connections are added 
    between the input and output of every hidden block. 

    - The (1x1) convolution-layer (sometimes referred to as depthwise convolution) can be used 
      for dimensionality reduction. This idea was popularized by the authors of the InceptionNet 
      architecture
    
    - Residual connections are used to achieve better gradient flow during training 
      This idea was popularized by the authors of the ResNET50 architecture
    """

    def __init__(
        self,
        num_of_filters: int,
        num_of_blocks: int = 3,
        kernel_size: tuple = (3,3),
        leaky_relu_slope: float = 0.10 ):

        """
        Description:
        Class Constructor.

        Arguments:
         - num_of_filters: (int) number of convolution kernels per convolution layer.
         - num_of_blocks: (int) number of Conv2D-BatchNorm-ReLU blocks. The default value is 3: 1 input block and 2 hidden blocks
         - kernel_size: (tuple of ints) kernel dimensions for hidden convolution blocks in pixels (height x width). The default is (3,3)
         - leaky_relu_slope: (float) ReLU slope for negative inputs. The default is 0.15 (A negative sign is implicitely assumed)
        """

        assert(num_of_filters > 0)
        assert(num_of_blocks > 1)
        assert(len(kernel_size) == 2)
        assert(kernel_size[0] > 0 and kernel_size[1] > 0)
        
        self._layer_name = "[Conv2D[1x1]*{input_depth}-BatchNorm-ReLU]->[Conv2D[{height}x{width}]*{hidden_depth}-BatchNorm-ReLU]*{blocks}".format(
            input_depth = num_of_filters,
            height = kernel_size[0], 
            width = kernel_size[1], 
            hidden_depth = num_of_filters, 
            blocks = num_of_blocks - 1
        )
        
        # super().__init__( name = self._layer_name )
        super().__init__()

        # Total number of Conv2D-BN-ReLU blocks
        self._num_of_blocks = num_of_blocks

        # Lists for storing Conv2D, BatchNorm, ReLU and Residual layer instances
        self._conv_layers = []
        self._batch_norm_layers = [ tf.keras.layers.BatchNormalization() for i in range(self._num_of_blocks) ]
        self._relu_layers = [ tf.keras.layers.LeakyReLU( alpha = abs(leaky_relu_slope)) for i in range(self._num_of_blocks) ]
        self._residual_layers = [ tf.keras.layers.Add() if i > 0 else None for i in range(self._num_of_blocks) ]

        # Optional Dropout layer (it is meant to be used after the last Conv2D-BN-ReLU block)
        self._dropout_rate = 0.05
        self._dropout_layer = tf.keras.layers.Dropout(rate = self._dropout_rate)
        
        for i in range(self._num_of_blocks):
            self._conv_layers.append(
                tf.keras.layers.SeparableConv2D(
                    filters = num_of_filters,
                    kernel_size = kernel_size if i > 0 else (1,1),
                    padding = "same"
                )
            )

    def call(self, inputs, training = False):
        """
        Description:
        The 'call' method defines the computation graph of this custom layer.
        The 'inputs' tensor first goes through the 1x1 Conv-BatchNorm-ReLU block. 
        The output of this computation is then fed to the hidden Conv-BatchNorm-ReLU blocks.
        With the exception of the first 1x1 conv block, residual connections are used to between
        the input and output of every hidden conv block. 

        Arguments List: 
        -> inputs: (4D tensor) {batch, height, width, channels}
        -> training: (bool) Set to True during training. Set to False during inference mode

        Returns: 
        -> output: (4D tensor) {batch, height, width, channels} The output of the final hidden block.
           * 'batch', 'height' and 'width' are equal to the corresponding dimensions of the input-tensor.
           * 'channels' is the equal to 'num_of_filters' argument provided in the constructor.
        """

        # Placeholder variable for the input tensor of the current Conv2D-BN-ReLU block
        previous_tensor = inputs

        # Placeholder variable for the output tensor of the current Conv2D-BN-ReLU block
        current_tensor = None

        # Define the computation graph of this custom-layer
        for i in range(self._num_of_blocks):

            # Conv2D -> BatchNorm -> ReLU
            current_tensor = self._conv_layers[i](previous_tensor)
            current_tensor = self._batch_norm_layers[i](current_tensor, training = training)
            current_tensor = self._relu_layers[i](current_tensor)

            # Apply the Residual connection (the first Conv2D-BN-ReLU block does not use one)
            if i > 0:
                current_tensor = self._residual_layers[i]([current_tensor, previous_tensor])

            # Save the output to use it as an input for the next iteration
            previous_tensor = current_tensor

        # Uncomment the following line to enable the optional Dropout layer
        # return self._dropout_layer(current_tensor, training=training)
        
        # Return the final output tensor
        return current_tensor

In [4]:
class ConvBlockLSTM(tf.keras.layers.Layer):
    
    """
    Description:
    This class implements the basic Conv-LSTM layer of the architecture. It consists
    of multiple, stacked ConvLSTM layers with additional Residual connections and 
    Batch-Normalization layers in-between. 
    """

    def __init__(
        self, 
        num_of_filters: int, 
        kernel_size: tuple = (3,3),
        num_of_layers: int = 1, 
        bidirectional: bool = False):
        
        """
        Class constructor: 

        Arguments List: 
        num_of_filters: (int) Number of filters per ConvLSTM layer.
        kernel_size: (int) pixel dimensions for convolution kernels {height x width}. The default is [3x3] pixels.
        num_of_layers: (int) Total number of stacked ConvLSTM layers. The default is 1 (single ConvLSTM layer)
        bidirectional: (bool) if True, use bidirectional ConvLSTM layers. The default is False (feedforward, unidirectional layers).
        """

        # Invoke the constructor of the base class (tf.keras.layers.Layer)
        super().__init__()

        # Placeholder attributes for the arguments of the constructor
        self._num_of_layers = num_of_layers
        self._conv_lstm_layers = [None] * self._num_of_layers
        self._batch_norm_layers = [None] * self._num_of_layers
        self._residual_layers = [None] * self._num_of_layers

        # Stack multiple ConvLSTM Layers. 
        for i in range(self._num_of_layers):

            # (Time-Distributed) Batch-Normalization layers
            self._batch_norm_layers[i] = tf.keras.layers.BatchNormalization()
            self._batch_norm_layers[i] = tf.keras.layers.TimeDistributed(self._batch_norm_layers[i])

            # ConvLSTM layers
            self._conv_lstm_layers[i] = tf.keras.layers.ConvLSTM2D(
                filters = num_of_filters,
                kernel_size = kernel_size, 
                padding = "same",
                return_sequences = True, 
                return_state = False, 
                dropout = 0.0,
                recurrent_dropout = 0.0,
            )

            # If the bidirectional option is enabled, wrap the ConvLSTM layer with the BiDirectional layer
            if bidirectional == True: 
                self._conv_lstm_layers[i] = tf.keras.layers.BiDirectional(
                    layer = self._conv_lstm_layers[i],
                    merge_mode = 'sum',
                    backward_layer = None           # TODO: Use a distinct ConvLSTM layer for the backward direction (maybe?)
                )

            # Residual connection (channel-wise addition) between the input and 
            # the output of the ConvLSTM layer.
            # The first ConvLSTM layer cannot have a residual connection, since we don't 
            # know in advance how many feature-maps its input tensor has.
            if i > 0: 
                self._residual_layers[i] = tf.keras.layers.Add()

    def call(self, inputs, training = False):
        
        """
        Description: 
        The 'call' method defines the computation graph of this custom layer. 

        Arguments List: 
        -> inputs:  (5D tensor) { batch, timestep, height, width, channels }
        -> training: (bool) 

        Returns: 
        -> output: (5D tensor) { batch, timestep, height, width, channels }
        """

        # Placeholder variable for the output of the previous iteration
        temp = inputs

        # Placeholder variable for the output of the current iteration
        output = None

        # Define the computation graph of the stacked ConvLSTM layer
        for i in range(self._num_of_levels):
            
            # Batch-Norm -> ConvLSTM 
            output = self._batch_norm_layers[i](temp, training = training)
            output = self._conv_lstm_layers[i](output, training = training)

            # All ConvLSTM layers use a Residual connection with the exception of the first one.
            if i > 0: 
                output = self._residual_layers[i]([output, temp])

            # Save the output tensor of this iteration and used it as input for the next one
            temp = output

        # Return the output tensor of the last iteration
        return output

In [ ]:
class BroadCast(tf.keras.layers.Layer):
    """
    """

    def __init__():
        pass

    def call(self, inputs, training = False):
        pass

In [3]:
class TemporalUNET(tf.keras.models.Model):

    """
    This class implements a modified version of the UNET architecture. 
    UNET networks fall under the Fully-Convolutional-Neural-Networks (FCNN)
    category, because they do not make use of Dense Layers. Instead, they rely 
    solely on convolution layers and they attempt to learn an image-to-image 
    mapping. 

    A typical UNET network consists of:
    1) an encoder block (contracting path) with alternating Convolution and Downsampling layers (or strided Convolutions in some cases)
    2) a decoder block (expanding path) with alternating Convolution and Upsampling layers (or Deconvolutions in some cases)
    3) skip connections between all encoder-decoder levels with compatible dimensions

    UNET models are widely used for semantic segmentation tasks, because they are usually able to 
    achieve good performance even in small datasets.

    This particular implementation simply adds ConvLSTM layers in parallel to all skip connections between 
    the encoder and the decoder levels thus making it possible to extract temporal features from
    sequences of satelite images at different resolution levels. The Convolution layers in the 
    encoder and decoder are left as is and they are applied independently for every timestep of the 
    input tensors. 
    """

    def __init__(
        self,
        num_of_levels: int, 
        num_of_filters: list, 
        conv_blocks_per_level: int = 3, 
        kernel_size: tuple = (3,3), 
        leaky_relu_slope: float = 0.10):

        """
        Class Constructor: 

        Arguments List: 
        -> num_of_levels: (int) Number of resolution levels in the encoder and decoder.
        -> num_of_filters: (list of ints) Number of convolution kernels at every resolution level
        -> conv_blocks_per_level: 
        -> kernel_size: (tuple of ints) Kernel dimensions in pixels [height x width]
        -> leaky_relu_slope: ReLU slope for negative inputs.
        """

        # Invoke the constructor of the base class.
        super().__init__()
        
        self._num_of_levels = num_of_levels
        
        # Time-Distributed,  2D-Convolution layers on the encoder path 
        self._encoder_conv_blocks_2D = [None] * self._num_of_levels

        # Time-Distributed, 2D-Convolution layers on the decoder path
        self._decoder_conv_blocks_2D = [None] * self._num_of_levels

        # 2D-Conv-LSTM layers between the encoder and decoder path 
        self._lstm_layers_2D = [None] * self._num_of_levels

        # Skip connections (depth-wise concatenation) on the decoder path
        self._concatenate_layers = [None] * self._num_of_levels
        
        # Downsampling layers on the encoder path 
        self._pooling_layers = [None] * self._num_of_levels

        # Upsampling layers on the decoder path 
        self._upsampling_layers = [None] * self._num_of_levels

        # Build the encoder network / contracting path 
        for i in range(self._num_of_levels): 
            self._encoder_conv_blocks_2D[i] = ConvBlock2D(
                num_of_filters = num_of_filters[i],
                num_of_blocks = conv_blocks_per_level, 
                kernel_size = kernel_size, 
                leaky_relu_slope = leaky_relu_slope                
            )

            # Wrap the previous Convolution Layer with the TimeDistributed layer to let Keras know 
            # that this convolution operation is meant to be applied independently for every frame
            # of the input sequence.
            self._encoder_conv_blocks_2D[i] = tf.keras.layers.TimeDistributed(self._encoder_conv_blocks_2D[i])

            # The last resolution level on the encoder path does not require a Pooling layer 
            if i != self._num_of_levels - 1: 
                self._pooling_layers[i] = tf.keras.layers.MaxPooling2D(pool_size = (2,2), padding = "valid")
                self._pooling_layers[i] = tf.keras.layers.TimeDistributed(self._pooling_layers[i])

        # Add Conv2D-LSTM layers and residual connections between the corresponding encoder and decoder levels.
        for i in range(self._num_of_levels):
            self._lstm_layers_2D[i] = ConvBlockLSTM(
                num_of_filters = num_of_filters[i], 
                kernel_size = kernel_size, 
                num_of_layers = 2, 
                bidirectional = False
            )

        # Build the decoder network / expanding path
        for i in range(self._num_of_levels): 
            self._decoder_conv_blocks_2D[i] = ConvBlock2D(
                num_of_filters = num_of_filters[i], 
                num_of_blocks = conv_blocks_per_level, 
                kernel_size = kernel_size, 
                leaky_relu_slope = leaky_relu_slope
            )

            # Wrap the previous Convolution Layer with the TimeDistributed layer, to let Keras know 
            # that this convolution operation is meant to be applied independently for every frame
            # of the input sequence.
            self._decoder_conv_blocks_2D[i] = tf.keras.layers.TimeDistributed(self._decoder_conv_blocks_2D[i])

            # The first resolution level on the decoder path does not require an Upsampling layer
            if i != 0: 
                self._upsampling_layers[i] = tf.keras.layers.UpSampling2D(size = (2,2), interpolation = "bilinear")
                self._upsampling_layers[i] = tf.keras.layers.TimeDistributed(self._upsampling_layers[i])

            # The last resolution level does not require a channel-wise, concatenation layer
            if i != self._num_of_levels - 1: 
                self._concatenate_layers[i] = tf.keras.layers.Concatenate(axis = -1)

        self._output_layer = tf.keras.layers.Conv3D(
            filters = 1,
            kernel_size = (3,3,3),
            padding = "same",
            activation = "sigmoid"
        )
    
    def call(self, inputs, training = False):
        
        """
        Description:
        The 'call' method describes the computation graph of a model.
        First, the input-stream of satellite images is processed by the 
        convolution layers of the encoder. Then, the output of every 
        encoder-stage is fed to the corresponding ConvLSTM layer. The 
        output of these ConvLSTM layers is then fed to the decoder of the
        architecture. Pooling and Upsampling connections are used to 
        connect the different resolution levels of the encoder and decoder
        blocks. Finally, a 3D Convolution layer is used to produce the 
        segmentation masks of the next timesteps.

        Arguments List: 
         -> inputs: (tensor) The input of the model. A 5D tensor with the following dimensions: {batch, timestep, height, width, channels}
         -> training: (bool) True indicates that the model is in 'training' mode whereas False indicates that the model is in 'inference' 
            mode. For most layers this makes no difference, however certain types of layers (such as batch-norm layers) need this information 
            to work as expected.

        Return List: 
         -> output: 5D tensor { batch x timesteps x height x width x 1 } Model predictions (future segmentation masks)
        """

        temp = inputs
        
        encoder_outputs = [None] * self._num_of_levels
        lstm_outputs = [None] * self._num_of_levels
        decoder_outputs = [None] * self._num_of_levels

        # Define the computation graph of the contracting path / encoder network
        for i in range(self._num_of_levels):             
            encoder_outputs[i] = self._encoder_conv_blocks_2D[i](temp, training = training)

            if i != self._num_of_levels - 1:
                temp = self._pooling_layers[i](encoder_outputs[i])
        
        # Define the computation graph of the ConvLSTM layers between the encoder and decoder
        for i in range(self._num_of_levels): 
            lstm_outputs[i] = self._lstm_layers_2D[i](encoder_outputs[i], training = training)
        
        # Define the computation graph of the expanding path / decoder network
        for i in range(self._num_of_levels - 1, -1, -1):
            
            temp = lstm_outputs[i]
            
            # Concatenation
            if i != self._num_of_levels - 1:
                temp = self._concatenate_layers[i]([lstm_outputs[i], decoder_outputs[i+1]])
            
            # Convolution 
            decoder_outputs[i] = self._decoder_conv_blocks_2D[i](temp, training = training)

            # Upsampling 
            if i != 0:
                decoder_outputs[i] = self._upsampling_layers[i](decoder_outputs[i])

        output_tensor = self._output_layer(decoder_outputs[0], training = training)
        
        return output_tensor

In [9]:
model = TemporalUNET(
    num_of_levels = 5, 
    num_of_filters = [8, 12, 16, 24, 32]
) 

model.build(input_shape=(None, 10, 512, 512, 2))

model.summary()

encoder path: 0
(None, 10, 512, 512, 8)
encoder path: 1
(None, 10, 256, 256, 12)
encoder path: 2
(None, 10, 128, 128, 16)
encoder path: 3
(None, 10, 64, 64, 24)
encoder path: 4
(None, 10, 32, 32, 32)

LSTM path: 0
(None, 10, 512, 512, 8)
LSTM path: 1
(None, 10, 256, 256, 12)
LSTM path: 2
(None, 10, 128, 128, 16)
LSTM path: 3
(None, 10, 64, 64, 24)
LSTM path: 4
(None, 10, 32, 32, 32)

Decoder path: 4
concat shape: (None, 10, 32, 32, 32)
conv shape: (None, 10, 32, 32, 32)
upsampling size: (None, 10, 64, 64, 32)
Decoder path: 3
concat shape: (None, 10, 64, 64, 56)
conv shape: (None, 10, 64, 64, 24)
upsampling size: (None, 10, 128, 128, 24)
Decoder path: 2
concat shape: (None, 10, 128, 128, 40)
conv shape: (None, 10, 128, 128, 16)
upsampling size: (None, 10, 256, 256, 16)
Decoder path: 1
concat shape: (None, 10, 256, 256, 28)
conv shape: (None, 10, 256, 256, 12)
upsampling size: (None, 10, 512, 512, 12)
Decoder path: 0
concat shape: (None, 10, 512, 512, 20)
conv shape: (None, 10, 512, 512,

In [13]:
def BinaryJaccardIndex(
    y_true, 
    y_pred, 
    epsilon = 1e-5, 
    axis = -1, 
    use_cloud_mask = True, 
    cloud_threshold = 0.5,
    num_of_prediction_steps = 1):
    
    """
    Description:
    The Jaccard Index (also known as Intersection-over-Union, IoU) is a 
    commonly used metric in image segmentation tasks. 

    Given two segmentation masks A and B, the IoU is defined as: 
      IoU(A,B) = intersection(A,B) / union(A,B)

    IoU values close to 1, indicate good overlap between A and B
    IoU values close to 0, indicate poor overlap between A and B

    Arguments List:
    -> y_true: (5D tensor) Ground-truth segmentation mask + optional cloud-probability masks { batch x timesteps x height x width x 2 }
    -> y_pred: (5D tensor) Predicted segmentation mask { batch x timesteps x height x width x 1 } 
    -> epsilon: (float) small positive constant for numeric stability in tensor division operations
    -> axis: (int)
    -> use_cloud_mask: (bool) if True, all pixels with a cloud-probability greater than 'cloud_threshold' are not taken into account
       The default is True
    -> cloud_threshold: (float) See above
    -> num_of_prediction_steps: (int) Number of future prediction steps to take into account when calculating this metric. The default
       is 1 which means that only the next prediction step is taken into account.

    Returns:
    -> IoU: scalar, 1D tensor

    References: 
    -> https://github.com/keras-team/keras-contrib/blob/master/keras_contrib/losses/jaccard.py
    """

    # Flatten tensors down to 1 dimension
    y_true = tf.keras.backend.flatten(y_true)
    y_pred = tf.keras.backend.flatten(y_pred)

    # Intersection (Overlap between ground-truth and predictions)
    intersection = tf.keras.backend.abs(y_true * y_pred)
    intersection = tf.keras.backend.sum(intersection, axis=axis)

    # Union (Required for scaling / normalization step)
    union = tf.keras.backend.abs(y_true) + tf.keras.backend.abs(y_pred)
    union = tf.keras.backend.sum(union, axis=axis)
    union = union - intersection

    return  intersection / (union + epsilon)

def BinaryJaccardLoss(
    y_true, 
    y_pred, 
    smoothing = 1e-5, 
    axis = -1, 
    use_cloud_mask = False, 
    num_of_prediction_steps = 1): 
    
    """
    Description: 
    The Jaccard Loss function is defined as: 
      Jac(A,B) = 1 - IoU(A,B)

    Since all keras optimizers work by minimizing a loss function, 
    the Jaccard Loss function should be used when attempting to 
    maximize the IoU metric of a prediction model.

    Arguments List: 
    -> y_true: (tensor)
    -> y_pred: (tensor) 
    -> smoothing: (float)
    -> axis: (int)
    -> use_cloud_mask: (bool) 
    -> num_of_prediction_steps: int

    Returns: 
    -> loss: scalar 1D tensor
    """

    # Flatten tensors down to 1 dimension
    y_true = tf.keras.backend.flatten(y_true)
    y_pred = tf.keras.backend.flatten(y_pred)

    # "Fuzzy" Intersection (Overlap between ground-truth and predictions)
    intersection = tf.keras.backend.abs(y_true * y_pred)
    intersection = tf.keras.backend.sum(intersection, axis=axis)

    # "Fuzzy" Union (Required for scaling / normalization step)
    union = tf.keras.backend.abs(y_true) + tf.keras.backend.abs(y_pred)
    union = tf.keras.backend.sum(union, axis=axis)
    union = union - intersection

    # Intersection-over-Union (Normalized overlap between ground-truth and predictions)
    IoU = (intersection + smoothing) / (union + smoothing) 

    return 1 - IoU 

def BinaryDiceCoefficient(
    y_true, 
    y_pred, 
    epsilon = 1e-5, 
    axis = -1, 
    use_cloud_mask = False, 
    num_of_prediction_steps = 1):
    
    """
    Description:
    Much like the Jaccard Index, the Dice coefficient is also used 
    in image segmentation tasks to measure the similarity between two
    segmentation maps. 

    Given two segmentation masks A and B, the Dice coefficient is defined as: 
      Dice(A,B) = 2 * intersection(A,B) / ( union(A,B) + intersection(A,B) )

    Values close to 1 indicate strong agreement between A and B, whereas 
    values close to 0 indicate poor agreement between A and B.

    Arguments List:
    -> y_true: (tensor)
    -> y_pred: (tensor)
    -> epsilon: (float)
    -> axis: (int)
    -> use_cloud_mask: (bool)

    Returns:
    -> coeff: scalar 1D tensor
    """

    # Flatten tensors down to 1 dimension
    y_true = tf.keras.backend.flatten(y_true)
    y_pred = tf.keras.backend.flatten(y_pred)

    # "Fuzzy" Intersection (Overlap between ground-truth and predictions)
    intersection = tf.keras.backend.abs(y_true * y_pred)
    intersection = tf.keras.backend.sum(intersection, axis = axis)

    # "Fuzzy" Union (Required for scaling / normalization step)
    union = tf.keras.backend.abs(y_true) + tf.keras.backend.abs(y_pred)
    union = tf.keras.backend.sum(union, axis = axis)
    union = union - intersection

    # Dice Coefficient 
    return ( 2 * intersection ) / ( union + intersection + epsilon )

def BinaryDiceLoss(
    y_true, 
    y_pred, 
    smoothing = 1e-5, 
    use_cloud_mask = False, 
    num_of_prediction_steps = 1):
    
    """
    Description: 
    The Dice Loss is defined as: 
      DiceLoss(A, B) = 1 - DiceCoeff(A,B)

    Since keras optimizers expect a loss function to minimize, 
    the Dice Loss function should be used when attempting to maximize
    the Dice coefficient of a prediction model.

    Arguments List:
    -> y_true: (tensor) Ground-truth segmentation masks as a binary tensor.
    -> y_pred: (tensor) Model predictions.
    -> smoothing: (float) 
    -> use_cloud_mask: (bool)

    Returns:
    -> loss: scalar 1D tensor
    """

    # Flatten tensors down to 1D vectors
    y_true = tf.keras.backend.flatten(y_true)
    y_pred = tf.keras.backend.flatten(y_pred)

    # "Fuzzy" Intersection (Overlap between ground-truth and predictions)
    intersection = tf.keras.backend.abs(y_true * y_pred)
    intersection = tf.keras.backend.sum(intersection, axis = axis)

    # "Fuzzy" Union (Required for normalization)
    union = tf.keras.backend.abs(y_true) + tf.keras.backend.abs(y_pred)
    union = tf.keras.backend.sum(union, axis = axis)
    union = union - intersection

    # Dice coefficient with smoothing factor
    dice = ( 2 * intersection + smoothing ) / ( union + intersection + smoothing ) 

    # Dice Loss
    return 1 - dice 

In [25]:
import numpy as np
from PIL import Image

imgs = np.random.randint(0, 255, (100, 50, 50, 3), dtype=np.uint8)

imgs[:,:,:,0] = imgs[:,:,:,0]
imgs[:,:,:,1] = imgs[:,:,:,0]
imgs[:,:,:,2] = imgs[:,:,:,0]

imgs = [Image.fromarray(img) for img in imgs]

# duration is the number of milliseconds between frames; this is 40 frames per second
imgs[0].save("array.gif", save_all=True, append_images=imgs[1:], duration=50, loop=0)

PIL.Image.Image

In [17]:
def GIF(
    arr, 
    cloud_mask = None, 
    label = 'mask'):
    
    temp = None
    
    if label == 'NDWI' or label == 'mNDWI' or label == 'NDVI':
        temp = (arr + 1.0) * 255
    elif label = 'VV' or label == 'VH':
        temp = ( (arr + 50) / (50 + 10) ) * 255
    elif label == 'mask':
        temp = np.copy(arr)
    else:
        raise ValueError("Invalid data format")

    temp[temp < 0] = 0
    temp[temp > 255] = 255

Help on function save in module PIL.Image:

save(self, fp, format=None, **params) -> 'None'
    Saves this image under the given filename.  If no format is
    specified, the format to use is determined from the filename
    extension, if possible.
    
    Keyword options can be used to provide additional instructions
    to the writer. If a writer doesn't recognise an option, it is
    silently ignored. The available options are described in the
    :doc:`image format documentation
    <../handbook/image-file-formats>` for each writer.
    
    You can use a file object instead of a filename. In this case,
    you must always specify the format. The file object must
    implement the ``seek``, ``tell``, and ``write``
    methods, and be opened in binary mode.
    
    :param fp: A filename (string), pathlib.Path object or file object.
    :param format: Optional format override.  If omitted, the
       format to use is determined from the filename extension.
       If a file object w

In [26]:
help(PIL.Image.Image.save)

Help on function save in module PIL.Image:

save(self, fp, format=None, **params) -> 'None'
    Saves this image under the given filename.  If no format is
    specified, the format to use is determined from the filename
    extension, if possible.
    
    Keyword options can be used to provide additional instructions
    to the writer. If a writer doesn't recognise an option, it is
    silently ignored. The available options are described in the
    :doc:`image format documentation
    <../handbook/image-file-formats>` for each writer.
    
    You can use a file object instead of a filename. In this case,
    you must always specify the format. The file object must
    implement the ``seek``, ``tell``, and ``write``
    methods, and be opened in binary mode.
    
    :param fp: A filename (string), pathlib.Path object or file object.
    :param format: Optional format override.  If omitted, the
       format to use is determined from the filename extension.
       If a file object w